In [16]:
import pandas as pd
import bokeh
import astropy
from astropy import units as u
from astropy import constants as c
from astropy.coordinates import SkyCoord
import json
from math import pi
import numpy as np

In [17]:
exos=pd.read_csv('exoplanets.csv')
exos[['RightAscension', 'Declination']].head()
print np.min(exos['DistFromSunParsec']), np.max(exos['DistFromSunParsec'])

1.3417 8500.0


In [48]:
def get_values(row):
    
    name=str(obj[1]['PlanetIdentifier'])
    jupiter_mass=possiblefail(obj,'PlanetaryMassJpt')
    earth_mass=tryingtoderive(jupiter_mass, earthmass)
    axis_au=possiblefail(obj,'SemiMajorAxisAU')
    disc_method=str(obj[1]['DiscoveryMethod'])
    disc_year=possiblefail(obj,'DiscoveryYear')
    ra=possiblefail(obj,'RightAscension')
    dec=possiblefail(obj,'Declination')
    distance=possiblefail(obj, 'DistFromSunParsec')
    host_mass=possiblefail(obj, 'HostStarMassSlrMass')
    host_temp=possiblefail(obj, 'HostStarTempK')
    host_lum=tryingtoderive(host_temp, lum)
    hostmag=tryingtoderive(host_lum, abs_mag)
    try:
        apparentmagnitude=app_mag(hostmag, distance)
    except ValueError:
        apparentmagnitude=5.0
    if apparentmagnitude==np.nan or apparentmagnitude=="None":
        apparentmagnitude=5.0
    return name, jupiter_mass, earth_mass, axis_au, disc_method, disc_year, ra, dec, distance, host_mass, apparentmagnitude
        
def possiblefail(obj, valuename):
    if pd.isnull(obj[1][valuename])==False:
        try:
            possiblevalue=obj[1][valuename]
        except ValueError:
            possiblevalue='None'
        return possiblevalue
    else:
        return np.nan

def tryingtoderive(value, func):
    try:
        new_val=func(value)
    except ValueError:
        new_val='None'
    return new_val

def earthmass(jup_mass):
    return 317.8*jup_mass

def lum(temp):
    return 4.*pi*c.R_sun.to(u.m).value**2*c.sigma_sb.value*temp**4

def abs_mag(lum):
    return -2.5*np.log10(lum/c.L_sun.value)

def app_mag(absmag, dist):
    if pd.isnull(dist)==True:
        return 5.
    else:
        return 5.*np.log10(dist)+absmag-5.

In [55]:
out=open('exos_radians.json','w')
#out.write('#name, apparentmagnitude, ra, dec, jupiter_mass, earth_mass, axis_au, disc_method, disc_year, distance, host_mass\n')
ss_planets=['Mercury','Venus','Earth','Mars','Jupiter','Saturn','Uranus', 'Neptune','Pluto']
exos_rad=[]
for obj in exos.iterrows():
    cont=True
    name, jupiter_mass, earth_mass, axis_au, disc_method, disc_year, ra, dec, distance, host_mass, apparentmagnitude=get_values(obj)
    for planet in ss_planets:
        if name==planet:
            cont=False
    if ra=='None' or dec=='None':
        cont=False
    if cont==True:
        if apparentmagnitude=="nan" or pd.isnull(apparentmagnitude)==True or apparentmagnitude>5.:
            apparentmagnitude=5.0
        elif apparentmagnitude<-0.5:
            apparentmagnitude=-0.5
        ra=ra.replace(' ',':')
        dec=dec.replace(' ',':')
        #print ra, dec
        s = SkyCoord(ra+' '+dec, unit=(u.hourangle, u.deg))
        l=[name, apparentmagnitude, s.ra.radian, s.dec.radian, jupiter_mass, earth_mass, axis_au, disc_method, disc_year, distance, host_mass]
        newl=[]
        for value in l:
            if pd.isnull(value)==True:
                value='null'
            newl.append(value)
        print newl
        exos_rad.append(newl)
out.write(json.dumps({'exos':exos_rad}))
out.close()
# Output values: name, apparentmagnitude, ra, dec, jupiter_mass, earth_mass, axis_au, disc_method, disc_year, distance, host_mass

['KOI-1843.03', 5.0, 4.974416715427641, 0.7019845151215094, 0.0014, 0.44492000000000004, 0.0048, 'transit', 2012.0, 'null', 0.46]
['KOI-1843.01', 5.0, 4.974416715427641, 0.7019845151215094, 'null', 'null', 0.039, 'transit', 'null', 'null', 0.46]
['KOI-1843.02', 5.0, 4.974416715427641, 0.7019845151215094, 'null', 'null', 0.052000000000000005, 'transit', 'null', 'null', 0.46]
['Kepler-9 b', 5.0, 4.98415128933064, 0.6702209771762558, 0.25, 79.45, 0.14300000000000002, 'transit', 2010.0, 650.0, 1.07]
['Kepler-9 c', 5.0, 4.98415128933064, 0.6702209771762558, 0.17, 54.026, 0.22899999999999998, 'transit', 2010.0, 650.0, 1.07]
['Kepler-9 d', 5.0, 4.98415128933064, 0.6702209771762558, 0.022000000000000002, 6.991600000000001, 0.0271, 'transit', 2010.0, 650.0, 1.07]
['GJ 160.2 b', 4.416992978046169, 1.0759112251759817, -0.3639556377352513, 0.0321, 10.201379999999999, 0.053, 'RV', 2014.0, 43.25, 0.69]
['Kepler-566 b', 5.0, 5.088580156241634, 0.8345194855674664, 'null', 'null', 'null', 'transit', 20